In [1]:
import numpy as np

def compute_topology_overlaps(G2, G4):
    """
    Computes the topological invariants I_22, I_24, I_44 based on set overlaps.
    I_alpha_beta counts how many sets share IDENTICAL elements.
    """
    # Helper to count identical sets
    def count_matches(list_a, list_b):
        matches = 0
        # Convert to sorted tuples to ensure order doesn't affect equality
        set_b = set(tuple(sorted(x)) for x in list_b)
        for item in list_a:
            if tuple(sorted(item)) in set_b:
                matches += 1
        return matches

    # For standard LABS/Ising chains, these overlaps are often 0 or specific integers
    # We implement the general counting logic here.
    I_22 = count_matches(G2, G2) # Self overlap is just len(G2)
    I_44 = count_matches(G4, G4) # Self overlap is just len(G4)
    I_24 = 0 # 2-body set vs 4-body set overlap usually 0 as sizes differ
    
    return {'22': I_22, '44': I_44, '24': I_24}

from math import sin, cos, pi

def compute_theta(t, dt, total_time, N, G2, G4):
    """
    Computes theta(t) using the analytical solutions for Gamma1 and Gamma2.
    """
    
    # ---  Better Schedule (Trigonometric) ---
    # lambda(t) = sin^2(pi * t / 2T)
    # lambda_dot(t) = (pi / 2T) * sin(pi * t / T)
    
    if total_time == 0:
        return 0.0

    # Argument for the trig functions
    arg = (pi * t) / (2.0 * total_time)
    
    lam = sin(arg)**2
    # Derivative: (pi/2T) * sin(2 * arg) -> sin(pi * t / T)
    lam_dot = (pi / (2.0 * total_time)) * sin((pi * t) / total_time)
    
    
    # ---  Calculate Gamma Terms (LABS assumptions: h^x=1, h^b=0) ---
    # For G2 (size 2): S_x = 2
    # For G4 (size 4): S_x = 4
    
    # Gamma 1 (Eq 16)
    # Gamma1 = 16 * Sum_G2(S_x) + 64 * Sum_G4(S_x)
    term_g1_2 = 16 * len(G2) * 2
    term_g1_4 = 64 * len(G4) * 4
    Gamma1 = term_g1_2 + term_g1_4
    
    # Gamma 2 (Eq 17)
    # G2 term: Sum (lambda^2 * S_x)
    # S_x = 2
    sum_G2 = len(G2) * (lam**2 * 2)
    
    # G4 term: 4 * Sum (4*lambda^2 * S_x + (1-lambda)^2 * 8)
    # S_x = 4
    # Inner = 16*lam^2 + 8*(1-lam)^2
    sum_G4 = 4 * len(G4) * (16 * (lam**2) + 8 * ((1 - lam)**2))
    
    # Topology part
    I_vals = compute_topology_overlaps(G2, G4)
    term_topology = 4 * (lam**2) * (4 * I_vals['24'] + I_vals['22']) + 64 * (lam**2) * I_vals['44']
    
    # Combine Gamma 2
    Gamma2 = -256 * (term_topology + sum_G2 + sum_G4)

    # ---  Alpha & Theta ---
    if abs(Gamma2) < 1e-12:
        alpha = 0.0
    else:
        alpha = - Gamma1 / Gamma2
        
    return dt * alpha * lam_dot

def test():
    @cudaq.kernel
    def two_qubit_layer(theta, q1, q2):
        rx(np.pi/2, q1)
        
        cx(q1, q2)
        rz(theta, q2)
        cx(q1, q2)
        
        rx(np.pi/2, qubits[q1])
        rx(-np.pi/2, q2)
        
        cx(q1, q2)
        rz(theta, q2)
        cx(q1, q2)
        
        rx(-np.pi/2, q1)

In [9]:
import cudaq
import time

n = 10

pairs = []
for i in range(1,n):
    square = []
    for j in range(n-i):
        square += [(i+j,j)]
    
    pairs += [square]

full = []
for x in range(len(pairs)):
    test = pairs[x]
    for i in range(len(test)):
        for j in range(len(test)):
            list1 = [test[i][0], test[i][1], test[j][0], test[j][1]]
            set1 = set()
            for w in list1:
                if list1.count(w) == 1:
                    set1.add(w)
            if set1 != set():
                full += [set1]

unique = [set(s) for s in set(frozenset(s) for s in full)]

list_pairs = []
for i in unique:
    list_pairs += [list(i)]

list_2 = []
list_4 = []
G2 = []
G4 = []

for i in list_pairs:
    if len(i) == 2:
        list_2 += [i[0], i[1]]
        G2 += [i]
    else:
        list_4 += [i[0], i[1], i[2], i[3]]
        G4 += [i]
    

T=1               # total time
n_steps = 1       # number of trotter steps
dt = T / n_steps
N = n

thetas =[]

for step in range(1, n_steps + 1):
    t = step * dt
    theta_val = compute_theta(t, dt, T, N, G2, G4)
    thetas.append(theta_val)


@cudaq.kernel
def qc(n: int, indices: list[int], indices2: list[int], theta_list: list[float]):
    qubits = cudaq.qvector(n)
    
    # Apply gates
    for i in range(n):
        h(qubits[i])

    for t in range(len(theta_list)):
        for i in range(len(indices) // 2):
            i1 = indices[2*i]
            i0 = indices[2*i + 1]
    
            rx(np.pi/2, qubits[i0])
    
            cx(qubits[i0], qubits[i1])
            rz(theta_list[t], qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            rx(-np.pi/2, qubits[i0])
            rx(np.pi/2, qubits[i1])
    
            cx(qubits[i0], qubits[i1])
            rz(theta_list[t], qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            rx(-np.pi/2, qubits[i1])
    
        for i in range(len(indices2) // 4):
            i0 = indices2[4*i]
            i1 = indices2[4*i + 1]
            i2 = indices2[4*i + 2]
            i3 = indices2[4*i + 3]
    
            rx(-np.pi/2, qubits[i0])
            ry(np.pi/2, qubits[i1])
            ry(-np.pi/2, qubits[i2])
    
            cx(qubits[i0], qubits[i1])
            rz(-np.pi/2, qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            cx(qubits[i2], qubits[i3])
            rz(-np.pi/2, qubits[i3])
            cx(qubits[i2], qubits[i3])
    
            rx(np.pi/2, qubits[i0])
            ry(-np.pi/2, qubits[i1])
            ry(np.pi/2, qubits[i2])
            rx(-np.pi/2, qubits[i3])
    
            rx(-np.pi/2, qubits[i1])
            rx(-np.pi/2, qubits[i2])
    
            cx(qubits[i1], qubits[i2])
            rz(theta_list[t], qubits[i2])
            cx(qubits[i1], qubits[i2])
    
            rx(np.pi/2, qubits[i1])
            rx(np.pi, qubits[i2])
    
            ry(np.pi/2, qubits[i1])
    
            cx(qubits[i0], qubits[i1])
            rz(np.pi/2, qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            rx(np.pi/2, qubits[i0])
            ry(-np.pi/2, qubits[i1])
    
            cx(qubits[i1], qubits[i2])
            rz(-theta_list[t], qubits[i2])
            cx(qubits[i1], qubits[i2])
    
            rx(np.pi/2, qubits[i1])
            rx(-np.pi, qubits[i2])
    
            cx(qubits[i1], qubits[i2])
            rz(-theta_list[t], qubits[i2])
            cx(qubits[i1], qubits[i2])
            
            rx(-np.pi, qubits[i1])
            ry(np.pi/2, qubits[i2])
    
            cx(qubits[i2], qubits[i3])
            rz(-np.pi/2, qubits[i3])
            cx(qubits[i2], qubits[i3])
    
            ry(-np.pi/2, qubits[i2])
            rx(-np.pi/2, qubits[i3])
    
            rx(-np.pi/2, qubits[i2])
    
            cx(qubits[i1], qubits[i2])
            rz(theta_list[t], qubits[i2])
            cx(qubits[i1], qubits[i2])
    
            rx(np.pi/2, qubits[i1])
            rx(np.pi/2, qubits[i2])
            
            ry(-np.pi/2, qubits[i1])
            ry(np.pi/2, qubits[i2])
    
            cx(qubits[i0], qubits[i1])
            rz(np.pi/2, qubits[i1])
            cx(qubits[i0], qubits[i1])
    
            cx(qubits[i2], qubits[i3])
            rz(np.pi/2, qubits[i3])
            cx(qubits[i2], qubits[i3])
    
            ry(np.pi/2, qubits[i1])
            ry(-np.pi/2, qubits[i2])
            rx(np.pi/2, qubits[i3])

    mz(qubits)


start = time.time()
result = cudaq.sample(qc, n, list_2, list_4, thetas, shots_count=100000)
end = time.time()
print(f"Time taken: {end - start} seconds")

# See results
#for bitstring, count in result.items():
    #print(f"{bitstring}: {count}")


# Measure
#mz(qubits)

Time taken: 0.7630367279052734 seconds
